In [1]:
import os

import pandas as pd

# 1. Read all CSV files into a single data frame

In [2]:
# Identify the location of the original files
# This represents the path: ../data/waitrose-2024-07
data_folder = os.path.join('..', 'data', 'waitrose-2024-07')

# Use a list comprehension to get all the files in the folder
all_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) 
             if file.endswith('.csv')]

# Print the list of files if you want to check
# print(all_files)

# Read every single file as a DataFrame
# Save the dataframes in a list
list_of_dfs = [pd.read_csv(file) for file in all_files]

# Use pd.concat to concatenate all the files into a single DataFrame
df = pd.concat(list_of_dfs)

# Check that we have all the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25418 entries, 0 to 1294
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data-product-id        25418 non-null  int64  
 1   data-product-name      25418 non-null  object 
 2   data-product-type      25418 non-null  object 
 3   data-product-on-offer  25418 non-null  bool   
 4   data-product-index     25408 non-null  float64
 5   image-url              25418 non-null  object 
 6   product-page           25418 non-null  object 
 7   product-name           25407 non-null  object 
 8   product-size           25363 non-null  object 
 9   item-price             25407 non-null  object 
 10  price-per-unit         24976 non-null  object 
 11  offer-description      7201 non-null   object 
 12  category               25418 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 2.5+ MB


# 2. Exploring the `item-price` column

**What did I learn?**

- Some numbers are in the format `£10.00` (with the pound symbol)
- Others are in the format `75p` (indicating pences)
- There might be others, I don't know.

How do I know? I tried to convert the string column to a float and got this error:

```python
# This is a way to validate if my column can be converted to float
# If the column is clean, this will not throw an error
df['item-price'].astype(float)
```

```python
ValueError: could not convert string to float: '£10.00'
```

Then, I looked at the column a bit further and discovered the `p` situation:

Possible solutions:

```python
float('£10.00'.replace('£', ''))
float('75p'.replace('p', '')) # But then multiply by 100
```


I found an alternative way to check for errors in the conversion using AI:

In [4]:
# Alternatively, I could check for errors like this:
pd.to_numeric(df['item-price'], errors='coerce').isna().sum()/len(df)

1.0

In [21]:
def clean_item_price(item_price: str): 
    # For example: item_price = '£1.99' --> '1.99'
    new_item_price = str(item_price).replace('£', '')
    return new_item_price


clean_item_price('£1.99')

'1.99'

In [24]:
df['item-price'].apply(clean_item_price)

0       10.00
1        9.00
2        8.00
3        4.00
4         75p
        ...  
1290     2.65
1291     4.00
1292     4.00
1293     4.75
1294     3.80
Name: item-price, Length: 25418, dtype: object

In [27]:
100 * (pd.to_numeric(df['item-price'].apply(clean_item_price), errors='coerce').isna().sum()/len(df))

4.583366118498701